In [1]:
import os

from ladybug.sql import SQLiteResult
from ladybug import analysisperiod as ap

from eppy import *

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import pandas as pd

import numpy as np

from collections import OrderedDict

import datetime as dt

from IPython.display import display
import kaleido



In [2]:
root = os.getcwd()
root

'/Users/julietnwagwuume-ezeoke/Documents/cee256_local/_final_project_local'

# SQL (Simulation) Analyis

In [3]:
def calc_metrics(simulated, measured):
    ts = simulated
    tm = measured
    tm_avg = np.mean(measured)
    n = len(tm)
    rmse = (100/tm_avg) * np.sqrt( (1/n) * np.sum((ts - tm)**2) )
    mbe = (100/tm_avg) * (np.sum(ts - tm)/n)
    return np.round(rmse,3), np.round(mbe,3)

In [4]:
def monthly_by_hour_from_sql(sql_dir_name):
    sqlFile = os.path.join(sql_dir_name, "eplusout.sql")
    sqld = SQLiteResult(sqlFile)

    elect = sqld.data_collections_by_output_name("Electricity:Facility")[-1]
    hot = sqld.data_collections_by_output_name("DistrictHeating:Facility")[-1]
    chill = sqld.data_collections_by_output_name("DistrictCooling:Facility")[-1]

    sim_month_by_hr = OrderedDict({
    "elect": elect.average_monthly_per_hour(),
    "hot": hot.average_monthly_per_hour(),
    "chill": chill.average_monthly_per_hour()
    })

    sim_monthly= OrderedDict({
    "elect": elect.total_monthly(),
    "hot": hot.total_monthly(),
    "chill": chill.total_monthly(),
    })

    return sim_month_by_hr, sim_monthly


# Load and Convert Historical Data

In [5]:
# conversion factors
tonHr_kwHr = 3.5169 # 1 ton-hr = 3.517 kWh
kbtu_kwHr = 1/3.412 # 1 kbtu = 0.29 kwHr

# load data 
historical_data_path = os.path.join(root, "data/Thornton Utilities 2019.xlsx")

# electricity # kwh 
hist_elect_hourly = pd.read_excel(historical_data_path, sheet_name="Electricity hourly", usecols="A:E")

# chilled water  -> convert ton hours to kWh
hist_chw_hourly = pd.read_excel(historical_data_path, sheet_name="CHW hourly", usecols="A:E")
hist_chw_hourly["kWh"] = hist_chw_hourly["ton-hours"]*tonHr_kwHr

# hot water -> convert kbtu to kWh
hist_hw_hourly = pd.read_excel(historical_data_path, sheet_name="HW hourly", usecols="A:E")
hist_hw_hourly["kWh"] = hist_hw_hourly["kBtu"]*kbtu_kwHr 

In [6]:
# month by hour
hist_month_by_hr = OrderedDict({
    "elect" : hist_elect_hourly.groupby(["Month", "Hour"])["kWh"].mean(),
    "hot": hist_hw_hourly.groupby(["Month", "Hour"])["kWh"].mean(),
    "chill": hist_chw_hourly.groupby(["Month", "Hour"])["kWh"].mean(),
})

# monthly 
hist_monthly = OrderedDict({
   "elect": hist_elect_hourly.groupby("Month")["kWh"].sum(),
   "hot":  hist_hw_hourly.groupby("Month")["kWh"].sum(),
   "chill":  hist_chw_hourly.groupby("Month")["kWh"].sum()
})

# Function Definition for Simulation Comparison

### hourly

In [103]:
hours = [dt.datetime(2019, 1, 1, i ).strftime("%H:%M") for i in range(24)]

def create_analysis_period(num):
    return ap.AnalysisPeriod(st_month=num, end_month=num)

def plot_datasets_monthly_by_hour(month_num, month_name):

    month = create_analysis_period(month_num)
    blueShades = px.colors.qualitative.Vivid[3:3+len(sim_data_hr.keys())]

    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=("Electricity", "Hot Water", "Chilled Water"))

    # historical/ measured data 
    for (ix, value) in  enumerate(hist_month_by_hr.values()):
        showLegend = True
        if ix > 0:
            showLegend = False
        fig.add_trace(go.Scatter(  
            x=hours,
            y=value[month.st_month] , 
            mode='lines',
            name="Historical",
            legendgroup="Months",
            line=dict(color='red'),
            showlegend=showLegend),
            row=1, col=ix+1)

    
    # simulated datasets 
    for (name, dataset), blueShade in zip(sim_data_hr.items(), blueShades):
        for (ix, sim), hist in zip(enumerate(dataset.values()),hist_month_by_hr.values()) :
            showLegend = True
            if ix > 0:
                showLegend = False
            fig.add_trace(go.Scatter( 
                x=hours,
                y=sim.filter_by_analysis_period(month).values , 
                mode='lines+markers',
                name=name,
                legendgroup="Months",
                line=dict(color=blueShade),
                showlegend=showLegend),
                row=1, col=ix+1)

    fig.update_layout(title=f"{month_name} Average Hourly Energy Usage",
        yaxis_title="Energy (kWh)",)
   
   # calculate metrics 
    metrics = {}
    for name, dataset in sim_data_hr.items():
        type_dict = {}
        for (k, sim), hist in zip(dataset.items(), hist_month_by_hr.values()):
            type_dict[k] = calc_metrics(sim.filter_by_analysis_period(month).values, hist[month.st_month])
        metrics[name] = type_dict
    metrics = pd.DataFrame(metrics)
    display(metrics)

    fig.show()

    return fig



### monthly 

In [83]:
months = [dt.datetime(2019, i+1, 1).strftime("%b") for i in range(12)]

def plot_datasets_monthly():
    blueShades = px.colors.qualitative.Vivid[:len(sim_data_hr.keys())]

    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=("Electricity", "Hot Water", "Chilled Water"))

    # historical/ measured data 
    for (ix, value) in  enumerate(hist_monthly.values()):
        showLegend = True
        if ix > 0:
            showLegend = False
        fig.add_trace(go.Scatter(  
            x=months,
            y=value, 
            mode='lines',
            name="Historical",
            legendgroup="Months",
            line=dict(color='red'),
            showlegend=showLegend),
            row=1, col=ix+1)

    # simulated datasets 
    for (name, dataset), blueShade in zip(sim_data_month.items(), blueShades):
        for (ix, sim), hist in zip(enumerate(dataset.values()),hist_monthly.values()) :
            showLegend = True
            if ix > 0:
                showLegend = False
            fig.add_trace(go.Scatter( 
                x=months,
                y=sim.values , 
                mode='lines+markers',
                name=name,
                legendgroup="Months",
                line=dict(color=blueShade),
                showlegend=showLegend),
                row=1, col=ix+1)

    fig.update_layout(title=f"Monthly Total Energy Usage",
        yaxis_title="Energy (kWh)",)
   
   # calculate metrics 
    metrics = {}
    for name, dataset in sim_data_month.items():
        type_dict = {}
        for (k, sim), hist in zip(dataset.items(), hist_monthly.values()):
            type_dict[k] = calc_metrics(sim.values, hist)
        metrics[name] = type_dict

    metrics_df = pd.DataFrame(metrics)
    display(metrics_df)

    fig.show()
    return fig, metrics

def saveImagePlotly(fig, fig_name):
    fig_name_png = fig_name + ".png"
    fig_path = os.path.join(root, "figs_final_pres_03_07", fig_name_png )
    fig.write_image(fig_path, format="png", engine="kaleido",  width=600, height=350, scale=2)


def pretty_metrics(metrics, fig_name):
    rmse = {}
    mbe = {}
    for model in metrics.values():
        rmse = {k: v[0] for k,v in model.items() }
        mbe = {k: v[1] for k,v in model.items() }
        split_metrics = pd.DataFrame([rmse, mbe])
        split_metrics.columns = ["Electricity", "Hot Water", "Chilled Water"]
        split_metrics["Metric"] = ["RMSE", "MBE"]
        # print(split_metrics)
        cols = [list(split_metrics.columns)[i] for i in [3,0,1,2]]
    
        fig = go.Figure(data=[go.Table(
        header=dict(values=cols,
                    # fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[split_metrics["Metric"], split_metrics["Electricity"], split_metrics["Hot Water"], split_metrics["Chilled Water"]],
                #    fill_color='lavender',
                   align='left'))
        ])

        fig.update_layout(
        {"plot_bgcolor": "rgba(0, 0, 0, 0)",
        "paper_bgcolor":"rgba(0, 0, 0, 0)",
        }, width=600,
        )
    saveImagePlotly(fig, fig_name)
    fig.show()

# Load SQL Files

In [116]:
# new_dir_name = os.path.join(adjusted_model_dir, "03-02_02-Hourly-Meter")
sim_names = {
        "Model 0": "final_pres_2_25",
        "Model 1": "final_pres_3_2",
        "Model 2": "final_pres_3_3",
        "Model 3": "final_pres_3_6"

}

adjusted_model_dir = os.path.join(root, "eppy_adjusted_models")
sim_name_paths = {k: os.path.join(adjusted_model_dir, v) for k,v in sim_names.items()}

sim_data_hr = OrderedDict()
sim_data_month = OrderedDict()

for k, v in zip(sim_names.keys(), sim_name_paths.values()):
    sim_data_hr[k] = monthly_by_hour_from_sql(v)[0]
    sim_data_month[k] = monthly_by_hour_from_sql(v)[1]



# Comparison of Annual Trends 

In [117]:
fig, metrics = plot_datasets_monthly()
model_num = 3
saveImagePlotly(fig, f"model{model_num}_monthly_e")

pretty_metrics(metrics, f"model{model_num}_metrics" )

,Model 0,Model 1,Model 2,Model 3
elect,"(25.543, 23.059)","(54.948, -53.987)","(41.964, -40.69)","(34.158, -32.478)"
hot,"(90.977, -79.78)","(83.017, -72.608)","(79.961, -69.692)","(84.888, -74.878)"
chill,"(166.736, 150.104)","(29.353, 14.816)","(66.133, 47.678)","(68.056, 51.115)"


# Comparison of Hourly Trends

In [118]:
# fig = plot_datasets_monthly_by_hour(1, "Jan")
# saveImagePlotly(fig, f"model3_hourly_e_jan")
# fig = plot_datasets_monthly_by_hour(6, "June")
# saveImagePlotly(fig, f"model3_hourly_e_june")